In [38]:
import sys
import nltk
import sklearn
import pandas
import numpy

import pandas as pd
import numpy as np
df = pd.read_csv('Bug Reports Dataset.csv')
print (df.head(5))

   Bug ID Product Component          Assignee    Status Classification  \
0  528905     JDT      Core    jdt-core-inbox  RESOLVED        Eclipse   
1  530231     JDT      Core    jdt-core-inbox  RESOLVED        Eclipse   
2  530758     JDT      Core          jarthana  VERIFIED        Eclipse   
3  531990     JDT      Core    jdt-core-inbox  VERIFIED        Eclipse   
4  532137     JDT      Core  register.eclipse  VERIFIED        Eclipse   

  Priority            Opened  \
0       P1  12/18/2017 11:46   
1       P1    1/24/2018 5:16   
2       P1    2/5/2018 20:38   
3       P1     3/5/2018 0:49   
4       P1    3/7/2018 16:17   

                                             Summary       Category  
0  JDT UI Gerrit failing with "invalid location f...  Configuration  
1  Compilation failure in M20180123-0400 in jdt.c...  Configuration  
2                    Build failure in I20180205-2000  Configuration  
3                    Build failure in I20180304-2000  Configuration  
4  Todays up

In [39]:
classes = df['Category']
print(classes.value_counts())

Program_Anomaly      498
GUI                  465
Test_Code            336
Performance          316
Configuration        281
NetworkOrSecurity    242
Name: Category, dtype: int64


In [40]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform (classes)
print(classes[:8])
print(Y[:8])
summary = df['Summary']
print(summary[:10])

0    Configuration
1    Configuration
2    Configuration
3    Configuration
4    Configuration
5    Configuration
6    Configuration
7    Configuration
Name: Category, dtype: object
[0 0 0 0 0 0 0 0]
0    JDT UI Gerrit failing with "invalid location f...
1    Compilation failure in M20180123-0400 in jdt.c...
2                      Build failure in I20180205-2000
3                      Build failure in I20180304-2000
4    Todays update produces compile error but javac...
5    Build 4.9.0 I-Build: I20180615-0300 failed due...
6    Cannot Start Eclipse after Plugin 'Ecore Diagr...
7                             Crash in parseXMLElement
8    org.mozilla.plugincontainer - EXC_BAD_ACCESS -...
9          English language pack not updated with 61.0
Name: Summary, dtype: object


In [41]:
import re
processed = re.sub(r'[\d]', ' ',str(summary))
print(processed)

        JDT UI Gerrit failing with "invalid location f...
        Compilation failure in M        -     in jdt.c...
                          Build failure in I        -    
                          Build failure in I        -    
        Todays update produces compile error but javac...
                              ...                        
        OpenH    Plugin does not work frequently in fi...
                  Crash in txMozillaXMLOutput::characters
        After restoring profile to a different locatio...
        User reports video artifact issues with Intel ...
        Slow shutdown of debug Firefox (~  s) builds a...
Name: Summary, Length:     , dtype: object


In [42]:
processed = summary.str.replace(r'(\d)' , '')
processed = processed.str.replace(r'^\D?(\d{3})\D?\D?(\d{3})\D?(\d{4})$' , ' ')
processed = processed.str.replace(r'\s+' , ' ')
processed = processed.str.replace(r'[ \t]+$' , ' ')
processed = processed.str.replace(r'[&#();*]' , ' ')
processed = processed.str.replace(r'[,:]' , ' ')
processed = processed.str.replace(r'[.!?\\-]' , ' ')
processed = processed.str.replace(r'[//]' , ' ')
processed = processed.str.replace(r'[|]' , ' ')
processed = processed.str.replace(r'\'' , ' ')
processed = processed.str.lower()
processed = processed.str.replace(r'\s[a-z]\s' , ' ')
processed = processed.str.replace(r'\[[^\]]*\]',' ')
processed = processed.str.replace(r'\"',' ')
processed = processed.str.replace(r'\`',' ')
print(processed)

0       jdt ui gerrit failing with  invalid location f...
1            compilation failure in  in jdt compiler tool
2                                       build failure in 
3                                       build failure in 
4       todays update produces compile error but javac...
                              ...                        
2133    openh plugin does not work frequently in firefox 
2134              crash in txmozillaxmloutput  characters
2135    after restoring profile to different location ...
2136    user reports video artifact issues with intel ...
2137    slow shutdown of debug firefox  ~s  builds aft...
Name: Summary, Length: 2138, dtype: object


In [43]:
from nltk.corpus import stopwords

In [44]:
stop_words = set(stopwords.words('english')) 

In [45]:
processed = processed.apply(lambda x: ' ' .join(term for term in x.split() if term not in stop_words))

In [46]:
print(processed)

0       jdt ui gerrit failing invalid location system ...
1                   compilation failure jdt compiler tool
2                                           build failure
3                                           build failure
4       todays update produces compile error javac com...
                              ...                        
2133                 openh plugin work frequently firefox
2134                  crash txmozillaxmloutput characters
2135    restoring profile different location paths ext...
2136    user reports video artifact issues intel gpu u...
2137    slow shutdown debug firefox ~s builds loading ...
Name: Summary, Length: 2138, dtype: object


In [47]:
#from nltk.stem.snowball import SnowballStemmer

In [48]:
#stemmer = SnowballStemmer("english")

In [49]:
#processed = processed.apply(lambda x: ' ' .join(stemmer.stem(term) for term in x.split()))

In [50]:
#print(processed)

In [51]:
from nltk.stem import WordNetLemmatizer 

In [52]:
lemmatizer = WordNetLemmatizer() 

In [53]:
processed = processed.apply(lambda x: ' ' .join(lemmatizer.lemmatize(term) for term in x.split()))

In [54]:
processed

0       jdt ui gerrit failing invalid location system ...
1                   compilation failure jdt compiler tool
2                                           build failure
3                                           build failure
4       today update produce compile error javac compi...
                              ...                        
2133                 openh plugin work frequently firefox
2134                   crash txmozillaxmloutput character
2135    restoring profile different location path exte...
2136    user report video artifact issue intel gpu upd...
2137    slow shutdown debug firefox ~s build loading h...
Name: Summary, Length: 2138, dtype: object

In [55]:
from nltk.tokenize import word_tokenize

In [56]:
allwords = []
for message in processed:
    words = word_tokenize(message)
    for w in words:
        allwords.append(w)
        
allwords = nltk.FreqDist(allwords)

print('Number of words: {}'.format(len(allwords)))
print('most common words: {}'.format(allwords.most_common(50)))

Number of words: 3720
most common words: [('test', 379), ('eclipse', 164), ('error', 141), ('intermittent', 135), ('failure', 121), ('org', 119), ('build', 113), ('>', 101), ('java', 99), ('memory', 98), ('j', 93), ('crash', 85), ('firefox', 84), ('plugin', 79), ('theme', 79), ('cpu', 77), ('dark', 72), ('button', 69), ('timed', 68), ('color', 68), ('ui', 67), ('tab', 67), ('http', 67), ('leak', 67), ('type', 67), ('exception', 64), ('page', 63), ('use', 62), ('window', 61), ('update', 59), ('high', 59), ('code', 59), ('file', 57), ('<', 56), ('using', 56), ('new', 55), ('fails', 53), ('nullpointerexception', 53), ('browser', 52), ('content', 51), ('internal', 51), ('assertion', 50), ('project', 50), ('method', 50), ('server', 49), ('usage', 49), ('freeze', 48), ('dom', 47), ('application', 47), ('can', 46)]


In [57]:
processed[0]

'jdt ui gerrit failing invalid location system library error'

In [58]:
word_features = allwords.most_common(50)
word_features = [i[0] for i in word_features]
print(word_features)

['test', 'eclipse', 'error', 'intermittent', 'failure', 'org', 'build', '>', 'java', 'memory', 'j', 'crash', 'firefox', 'plugin', 'theme', 'cpu', 'dark', 'button', 'timed', 'color', 'ui', 'tab', 'http', 'leak', 'type', 'exception', 'page', 'use', 'window', 'update', 'high', 'code', 'file', '<', 'using', 'new', 'fails', 'nullpointerexception', 'browser', 'content', 'internal', 'assertion', 'project', 'method', 'server', 'usage', 'freeze', 'dom', 'application', 'can']


In [59]:
def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)
    return features

features = find_features(processed[0])

for key, value in features.items():
    if value == True:
        print(key)

error
ui


In [60]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [61]:
vectorizer = TfidfVectorizer()

In [62]:
X = vectorizer.fit_transform(processed).toarray()

In [63]:
print(X.shape)

(2138, 3670)


In [64]:
pd.DataFrame(X, columns=vectorizer.get_feature_names()) 

,__cfrunloopservicemachport,__format,__nscfdictionary,_cairo_array_allocate,_psutil_osx,_ssl,_tests,aa,aarch,ab,...,youtube,yubikey,zero,zerohedge,zhihu,zlib,zombie,zoom,zoomed,zooming
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
Y = df['Category']

In [66]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [67]:
print(Y.value_counts())

Program_Anomaly      498
GUI                  465
Test_Code            336
Performance          316
Configuration        281
NetworkOrSecurity    242
Name: Category, dtype: int64


In [68]:
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)  
text_classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [69]:
predictions = text_classifier.predict(X_test)

In [70]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

[[43  1  1  1  8  0]
 [ 0 80  1  1  1  1]
 [ 2  1 40  1  7  2]
 [ 2  0  1 68  5  0]
 [ 0  4  0  0 94  1]
 [ 1  3  0  1  2 55]]
                   precision    recall  f1-score   support

    Configuration       0.90      0.80      0.84        54
              GUI       0.90      0.95      0.92        84
NetworkOrSecurity       0.93      0.75      0.83        53
      Performance       0.94      0.89      0.92        76
  Program_Anomaly       0.80      0.95      0.87        99
        Test_Code       0.93      0.89      0.91        62

         accuracy                           0.89       428
        macro avg       0.90      0.87      0.88       428
     weighted avg       0.89      0.89      0.89       428

0.8878504672897196


In [71]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

In [72]:
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))

[[37  3  5  2  5  2]
 [ 3 72  1  4  2  2]
 [ 7  5 27  5  3  6]
 [ 3 11  8 47  5  2]
 [10  9  6  1 71  2]
 [14  3  1  6  5 33]]
                   precision    recall  f1-score   support

    Configuration       0.50      0.69      0.58        54
              GUI       0.70      0.86      0.77        84
NetworkOrSecurity       0.56      0.51      0.53        53
      Performance       0.72      0.62      0.67        76
  Program_Anomaly       0.78      0.72      0.75        99
        Test_Code       0.70      0.53      0.61        62

         accuracy                           0.67       428
        macro avg       0.66      0.65      0.65       428
     weighted avg       0.68      0.67      0.67       428

0.6705607476635514


In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(solver='liblinear', random_state=0)
#fit the model
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)

In [74]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred_test))  
print(classification_report(y_test,y_pred_test))  
print(accuracy_score(y_test, y_pred_test))

[[36  6  0  2 10  0]
 [ 1 82  0  0  0  1]
 [ 0  3 36  1 12  1]
 [ 1  2  2 64  7  0]
 [ 0  4  0  0 94  1]
 [ 1  3  0  0  4 54]]
                   precision    recall  f1-score   support

    Configuration       0.92      0.67      0.77        54
              GUI       0.82      0.98      0.89        84
NetworkOrSecurity       0.95      0.68      0.79        53
      Performance       0.96      0.84      0.90        76
  Program_Anomaly       0.74      0.95      0.83        99
        Test_Code       0.95      0.87      0.91        62

         accuracy                           0.86       428
        macro avg       0.89      0.83      0.85       428
     weighted avg       0.87      0.86      0.85       428

0.8551401869158879


In [75]:
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(criterion = 'entropy').fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 
print("Accuracy:",metrics.accuracy_score(y_test, dtree_predictions))

Accuracy: 0.8387850467289719


In [76]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,dtree_predictions))  
print(classification_report(y_test,dtree_predictions))  
print(accuracy_score(y_test, dtree_predictions))

[[42  3  2  2  4  1]
 [ 3 71  1  3  5  1]
 [ 2  4 39  2  5  1]
 [ 2  2  2 66  4  0]
 [ 1  8  1  0 87  2]
 [ 2  4  1  1  0 54]]
                   precision    recall  f1-score   support

    Configuration       0.81      0.78      0.79        54
              GUI       0.77      0.85      0.81        84
NetworkOrSecurity       0.85      0.74      0.79        53
      Performance       0.89      0.87      0.88        76
  Program_Anomaly       0.83      0.88      0.85        99
        Test_Code       0.92      0.87      0.89        62

         accuracy                           0.84       428
        macro avg       0.84      0.83      0.84       428
     weighted avg       0.84      0.84      0.84       428

0.8387850467289719
